## UC1PR2101 - Train System
##### SUBMITTED BY: THOMAS THAULOW STÖCKLIN



In [1]:
import sqlite3
import traceback
import pandas as pd
from datetime import datetime, date, timedelta
conn = sqlite3.connect('jprr_simpified.sqlite')
cursor = conn.cursor()
cursor.execute('PRAGMA foreign_keys = ON;')
print("Created and/or Opened database successfully")
cursor.close()

Created and/or Opened database successfully


# 2- DDL and Database Creation #


#### 1- Create tables for the database ####


In [2]:
try:
    cursor = conn.cursor()
    cursor.executescript('''


        /* Create Table for TERMINALS */
        CREATE TABLE IF NOT EXISTS terminals (
        terminal_id INTEGER PRIMARY KEY AUTOINCREMENT,
        terminal_name TEXT NOT NULL,
        terminal_description TEXT,
        terminal_tracklength TEXT NOT NULL,
        table_constraints
        );
        
        /* Create Table for TERM_LOCATIONS */
        CREATE TABLE IF NOT EXISTS term_locations (
        terminal_id INTEGER NOT NULL,
        terminal_latitude TEXT NOT NULL,
        terminal_longitude TEXT NOT NULL,
        table_constraints,
        FOREIGN KEY (terminal_id) REFERENCES TERMINALS(terminal_id)
        );
        
        /* Create Table for ORIGIN*/
        CREATE TABLE IF NOT EXISTS origin (
        origin_Id INTEGER PRIMARY KEY AUTOINCREMENT,
        terminal_id INTEGER NOT NULL,
        departure_datetime TEXT NOT NULL,
        table_constraints,
        FOREIGN KEY (terminal_id) REFERENCES TERMINALS(terminal_id)
        );

        /* Create Table for DESTINATION*/
        CREATE TABLE IF NOT EXISTS destination (
        destination_id INTEGER PRIMARY KEY AUTOINCREMENT,
        terminal_id INTEGER NOT NULL,
        arrival_datetime TEXT NOT NULL,
        table_constraints,
        FOREIGN KEY (terminal_id) REFERENCES TERMINALS(terminal_id)
        );
       
        /* Create Table for CUSTOMERS */
        CREATE TABLE IF NOT EXISTS customers (
        cust_id INTEGER PRIMARY KEY AUTOINCREMENT,
        cust_name TEXT NOT NULL,
        cust_streetaddr TEXT NOT NULL,
        cust_city TEXT NOT NULL,
        cust_state TEXT NOT NULL,
        cust_zip TEXT NOT NULL,
        cust_phone TEXT NOT NULL,
        table_constraints
        );
        
        /* Create Table for CAR_TYPES */
        CREATE TABLE IF NOT EXISTS car_types (
        car_type_id INTEGER PRIMARY KEY AUTOINCREMENT,
        car_type TEXT NOT NULL,
        table_constraints
        );

        /* Create Table for CARS */
        CREATE TABLE IF NOT EXISTS cars (
        car_id INTEGER PRIMARY KEY AUTOINCREMENT,
        car_type_id INTEGER NOT NULL,
        car_weight INTEGER NOT NULL default 50000,
        car_capacity REAL NOT NULL default 50000,
        car_current_capacity REAL default 0,
        car_length TEXT NOT NULL,
        car_builddate TEXT NOT NULL,
        table_constraints,
        FOREIGN KEY (car_type_id) REFERENCES car_types(car_type_id)
        );

        /* Create Table for TRAINS */
        CREATE TABLE IF NOT EXISTS trains (
        train_id INTEGER PRIMARY KEY AUTOINCREMENT,
        loc_Id INTEGER NOT NULL,
        car_id INTEGER NOT NULL,
        table_constraints,
        FOREIGN KEY (loc_id) REFERENCES locomotives(loc_id),
        FOREIGN KEY (car_id) REFERENCES cars(car_id)        
        );

        /* Create Table for TRAIN_JOURNEY */
        CREATE TABLE IF NOT EXISTS train_journey (
        journey_id INTEGER PRIMARY KEY AUTOINCREMENT,
        loc_id INTEGER NOT NULL,
        train_id INTEGER NOT NULL,
        origin_id INTEGER NOT NULL,
        destination_id INTEGER NOT NULL,
        table_constraints,
        FOREIGN KEY (train_id) REFERENCES trains(train_id),
        FOREIGN KEY (loc_id) REFERENCES locomotives(loc_id),
        FOREIGN KEY (origin_id) REFERENCES origin(origin_id),
        FOREIGN KEY (destination_id) REFERENCES destination(destination_id)
        );

        /* Create Table for JOURNEY_CUSTOMER */
        CREATE TABLE IF NOT EXISTS journey_customer (
        journey_cust_id INTEGER PRIMARY KEY AUTOINCREMENT,
        journey_id INTEGER NOT NULL,
        cust_id INTEGER NOT NULL,
        table_constraints,
        FOREIGN KEY (journey_id) REFERENCES train_journey(journey_id),
        FOREIGN KEY (cust_id) REFERENCES customers(cust_id)
        );

        /* Create Table for SHIPMENTS */
        CREATE TABLE IF NOT EXISTS shipments (
        shipment_id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipment_item TEXT NOT NULL,
        shipment_description TEXT,
        shipment_weight INTEGER NOT NULL,
        shipment_volume REAL NOT NULL,
        shipment_price REAL NOT NULL,
        table_constraints
        );
        
        /* Create Table for SHIPMENT_JOURNEY */
        CREATE TABLE IF NOT EXISTS shipment_journey (
        shipment_journey_id INTEGER PRIMARY KEY AUTOINCREMENT,
        shipment_id INTEGER NOT NULL,
        train_id INTEGER NOT NULL,
        journey_cust_id INTEGER NOT NULL,
        table_constraints,
        FOREIGN KEY (train_id) REFERENCES trains(train_id),
        FOREIGN KEY (shipment_id) REFERENCES shipments(shipment_id),
        FOREIGN KEY (journey_cust_id) REFERENCES journey_customer(journey_cust_id)
        );

        /* Create Table for LOCOMOTIVES */
        CREATE TABLE IF NOT EXISTS locomotives (
        loc_id INTEGER PRIMARY KEY AUTOINCREMENT,
        loc_manufacturer TEXT NOT NULL,
        loc_model TEXT NOT NULL,
        loc_hp TEXT NOT NULL,
        loc_weight INTEGER NOT NULL,
        loc_builddate TEXT NOT NULL,
        table_constraints
        );

        ''')
    # cursor.close()

    print("Success. All tables are present")

except Exception: 
    print("Failed to create database. Please contact support")
    traceback.print_exc()     

Success. All tables are present


#### 2- Confirm table creation ####


In [3]:
schema = '''

    SELECT * FROM sqlite_master
    WHERE type='table' 
    ORDER BY rootpage;
     '''

cursor = conn.cursor()
cursor.execute( schema )
names = [ x[0] for x in cursor.description]
rows = cursor.fetchall()
main_table = pd.DataFrame( rows, columns=names)
main_table

,type,name,tbl_name,rootpage,sql
0,table,terminals,terminals,2,CREATE TABLE terminals (\n terminal_id ...
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,term_locations,term_locations,4,CREATE TABLE term_locations (\n termina...
3,table,origin,origin,5,CREATE TABLE origin (\n origin_Id INTEG...
4,table,destination,destination,6,CREATE TABLE destination (\n destinatio...
5,table,customers,customers,7,CREATE TABLE customers (\n cust_id INTE...
6,table,car_types,car_types,8,CREATE TABLE car_types (\n car_type_id ...
7,table,cars,cars,9,CREATE TABLE cars (\n car_id INTEGER PR...
8,table,trains,trains,10,CREATE TABLE trains (\n train_id INTEGE...
9,table,train_journey,train_journey,11,CREATE TABLE train_journey (\n journey_...


#### 3- Find schema data on tables ####

In [27]:
user_input = input("Select Table: ")

table_data = '''
    PRAGMA table_info('{}')   
     '''

cursor = conn.cursor()
cursor.execute( table_data.format(user_input) )
names = [ x[0] for x in cursor.description]
rows = cursor.fetchall()
Temp = pd.DataFrame( rows, columns=names)
Temp
    

Select Table: train_journey


,cid,name,type,notnull,dflt_value,pk
0,0,journey_id,INTEGER,0,None,1
1,1,loc_id,INTEGER,1,None,0
2,2,train_id,INTEGER,1,None,0
3,3,origin_id,TEXT INTEGER,0,None,0
4,4,destination_id,INTEGER,1,None,0
5,5,table_constraints,,0,None,0


#### 4- Confirm foreign keys ####


In [105]:
user_input = input("Enter table: ")

table_data = '''
    PRAGMA foreign_key_list('{}')                      
     '''

cursor = conn.cursor()
cursor.execute( table_data.format(user_input) )
names = [ x[0] for x in cursor.description]
rows = cursor.fetchall()
Temp = pd.DataFrame( rows, columns=names)
Temp

Enter table: trains


,id,seq,table,from,to,on_update,on_delete,match
0,0,0,cars,car_id,car_id,NO ACTION,NO ACTION,NONE
1,1,0,locomotives,loc_Id,loc_id,NO ACTION,NO ACTION,NONE


#### 5- Find Content in tables ####


In [72]:
user_input = input("Select Table Name: ")

table_data = '''
    SELECT *          
    FROM {}                      
     '''

cursor = conn.cursor()
cursor.execute(table_data.format(user_input))
names = [ x[0] for x in cursor.description]
rows = cursor.fetchall()
Temp = pd.DataFrame( rows, columns=names)
Temp

Select Table Name: destination


,destination_id,terminal_id,arrival_datetime,table_constraints
0,1,2,2020-08-12 21:04:20,None
1,2,2,2020-08-12 21:04:20,None


# 3- Stored Procedures #


#### STORED PROCEDURE 1 ####

In [3]:
def add_customer(customer):
  """
    To add a new Customer in the DB
    @params: Customer: Tuple (cust_name, address, city, state, zip, phone)
    @return: last inserted cust_name, if failed return NONE 
  """
  try:
    cursor = conn.cursor()
    SQL = 'INSERT INTO Customers (cust_name, cust_streetaddr, cust_city, cust_state, cust_zip, cust_phone) VALUES (?,?,?,?,?,?)'
    cursor.execute(SQL,customer)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
  
# print(add_customer(("Thomas Thaulow","Tjernsrudveien 43A","Stabekk","Viken","1368","40049254")))


def create_locomotives(locomotive):
  """
    To add a new Locomotive in the DB
    @params: Locomotive: Tuple (loc_manufacturer, loc_model, loc_hp, loc_weight, loc_builddate)
    @return: last inserted loc_id 
  """
  try:
    SQL = 'INSERT INTO locomotives (loc_manufacturer, loc_model, loc_hp, loc_weight, loc_builddate) VALUES (?,?,?,?,?)'
    cursor.execute(SQL,locomotive)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None

# print(create_locomotives(("Porche", "Superspeed", 2000, 20, datetime.now())))


def create_carTypes(car_type):
  """
    To add a new car_type in the DB
    @params: car_type: String(car_type)
    @return: last inserted loc_id 
  """
  try:
    SQL = "INSERT INTO car_types (car_type) VALUES (?)"
    cursor.execute(SQL,(car_type,))
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print("Create Car Types: ",e)
    return None

# print(create_carTypes("Flatcar"))



def create_cars(car):
  """
    To add a new Car in the DB
    @params: car: Tuple (car_type_id, car_weight, car_capacity, car_length, car_builddate)
    @return: last inserted loc_id 
  """
  try:
    SQL = 'INSERT INTO cars (car_type_id, car_weight, car_capacity, car_length, car_builddate) VALUES (?,?,?,?,?)'
    cursor.execute(SQL,car)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None

# print(create_cars((1, 12, 20, 49,"2020-08-12")))

def create_shipments(shipment):
  """
    To add a new Shipment in the DB
    @params: shipment: Tuple (shipment_item, shipment_description, shipment_weight, shipment_volume, shipment_price)
    @return: last inserted  shipment_id
  """
  try:
    SQL = 'INSERT INTO shipments ( shipment_item, shipment_description, shipment_weight, shipment_volume, shipment_price) VALUES (?,?,?,?,?)'
    cursor.execute(SQL,shipment)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_shipments(("Gold", "Lots of gold", 200, 40, 2500)))

def create_terminal(terminal):
  """
    To add a new terminal in the DB
    @params: terminal: Tuple (terminal_name, terminal_description, terminal_tracklength)
    @return: last inserted terminal_id 
  """
  try:
    SQL = 'INSERT INTO terminals (terminal_name, terminal_description, terminal_tracklength) VALUES (?,?,?)'
    cursor.execute(SQL,terminal)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_terminal(("Kristiansand Terminal", "A terminal in Kristiansand city", 4608)))

def create_train(train):
  """
    To add a new train in the DB
    @params: train: Tuple (loc_id, car_id)
    @return: last inserted train_id 
  """
  try:
    SQL = 'INSERT INTO trains (loc_id, car_id) VALUES (?,?)'
    cursor.execute(SQL,train)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_train((1,1)))


def create_destination(destination):
  """
    To add a new destination in the DB
    @params: destination: Tuple (terminal_id, arrival_datetime)
    @return: last inserted terminal_id 
  """
  try:
    SQL = 'INSERT INTO destination (terminal_id, arrival_datetime) VALUES (?,?)'
    cursor.execute(SQL,destination)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_destination((2, "2020-09-15 12:04:20")))


def create_origin(origin):
  """
    To add a new origin in the DB
    @params: origin: Tuple (terminal_id, departure_datetime)
    @return: last inserted journey_id 
  """
  try:
    SQL = 'INSERT INTO origin (terminal_id, departure_datetime) VALUES (?,?)'
    cursor.execute(SQL,origin)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_origin((1, "2020-08-12 12:16:55")))


def create_train_journey(train_journey):
  """
    To add a new train_journey in the DB
    train departure time should have an 8 hours gap from previous trip 
    @params: train_journey: Tuple (loc_id,train_id, origin_id, destination_id)
    @return: last inserted journey_id 
  """
  try:
    origin = """
      SELECT origin.departure_datetime from origin 
      where origin.origin_id = ? 
    """
    cursor.execute(origin,(train_journey[1],))
    conn.commit()
    originD = cursor.fetchone()
    # print(originD)

    destination = """ select arrival_datetime from destination where destination.destination_id = ? """
    cursor.execute(destination,(train_journey[2],))
    conn.commit()
    destinationD = cursor.fetchone()
    # print(destinationD)
    
    if originD == None or destinationD == None:
      return None
    # print(datetime.strptime(originD[0],"%Y-%m-%d %H:%M:%S")+timedelta(hours = 8))
    # print(datetime.strptime(originD[0],"%Y-%m-%d %H:%M:%S")+timedelta(hours = 8) < datetime.strptime(destinationD[0],"%Y-%m-%d %H:%M:%S"))
    # print(datetime.strptime(destinationD[0],"%Y-%m-%d %H:%M:%S"))
    
    if datetime.strptime(originD[0],"%Y-%m-%d %H:%M:%S")+timedelta(hours = 8) < datetime.strptime(destinationD[0],"%Y-%m-%d %H:%M:%S"):
      SQL = 'INSERT INTO train_journey (loc_id,train_id, origin_id, destination_id) VALUES (?,?,?,?)'
      cursor.execute(SQL,train_journey)
      conn.commit()
      return cursor.lastrowid
    else:
      print("Train should have atleast 8 hours Gap b/w Arrival and Departure")
      return None
  except Exception as e:
    print(e)
    return None
    
# print(create_train_journey((1, 1, 1)))

def create_shipment_journey(shipment_journey):
  """
    To add a new shipment_journey in the DB
    @params: shipment_journey: Tuple (journey_cust_id,train_id, shipment_id)
    @return: last inserted journey_id 
  """
  try:
    SQL = 'INSERT INTO shipment_journey (journey_cust_id, train_id ,shipment_id) VALUES (?,?,?)'
    cursor.execute(SQL,shipment_journey)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_shipment_journey((1, 1)))

def create_journey_customer(journey_customer):
  """
    To add a new journey_customer in the DB
    @params: journey_customer: Tuple (journey_id, cust_id)
    @return: last inserted journey_id 
  """
  try:
    SQL = 'INSERT INTO journey_customer (journey_id, cust_id) VALUES (?,?)'
    cursor.execute(SQL,journey_customer)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_journey_customer((1, 1)))

def create_term_location(term_location):
  """
    To add a new term_location in the DB
    @params: term_location: Tuple (terminal_id, terminal_latitude, terminal_longitude)
    @return: last inserted terminal_id 
  """
  try:
    SQL = 'INSERT INTO term_locations (terminal_id, terminal_latitude, terminal_longitude) VALUES (?,?,?)'
    cursor.execute(SQL,term_location)
    conn.commit()
    return cursor.lastrowid
  except Exception as e:
    print(e)
    return None
    
# print(create_term_location((1, 1, 1)))



In [4]:
create_locomotives(("Porche", "Superspeed", 2000, 20, "2012-05-02"))
create_locomotives(("BMW", "Superslow", 2010, 30, "2012-07-02"))
create_carTypes("flatcar")
create_cars((1, 50, 50000, 49,"2020-08-12"))
create_cars((1, 600, 50000, 49,"2020-09-12"))
create_terminal(("Stavanger", "A terminal in Kristiansand city", 1212))
create_terminal(("Kristiansand", "B terminal in Kristiansand city", 1212))
create_terminal(("Oslo", "C terminal in Kristiansand city", 1212))
create_train((1,1))
create_train((1,2))
create_origin((1, "2020-08-12 12:16:55"))
create_destination((2, "2020-08-12 21:04:20"))
create_train_journey((1,1, 1, 1))


#add_customer(("Thomas Thaulow","Tjernsrudveien 43a","Stabekk","Viken","1368","40049254"))
#create_shipments(("Gold", "Lots of gold", 200, 40, 2500))
#create_shipment_journey((1, 1, 1))

#create_journey_customer((1, 1))

create_term_location((1, 1, 1))

1

In [5]:
#Find functions
def find_train(source, destination, departure_date_time):
  '''
    To find a train
    @params: source: String, destination: String,  departure: DateTime
    @return: List<Trains>
  '''
  try:
    SQL = '''
    SELECT t1.terminal_name as origin_tName,t2.terminal_name as dest_tName,origin.departure_datetime,destination.arrival_datetime,cars.*,car_types.car_type,train_journey.journey_id  from train_journey 
    left join origin on train_journey.origin_id = origin.origin_id 
    left join terminals as t1 on origin.terminal_id = t1.terminal_id 
    left join destination on train_journey.destination_id = destination.destination_id 
    left join terminals as t2 on destination.terminal_id = t2.terminal_id 
    left join trains on train_journey.train_id = trains.train_id 
    left join cars on cars.car_id = trains.car_id 
    left join car_types on cars.car_type_id = car_types.car_type_id 
    where origin_tName = ? and dest_tName = ? and strftime('%Y-%m-%d %H:%M',departure_datetime) = strftime('%Y-%m-%d %H:%M',?)
    '''
    # SQL = "SELECT  *,?,?, strftime('%Y-%m-%d %H:%M',?)  from origin"

    cursor.execute(SQL,(source,destination,departure_date_time))
    conn.commit()
    rows = cursor.fetchall()
    # print(rows)
    names = [ x[0] for x in cursor.description]
    trains = {}
    trainObj = list()
    i=0;
    # print(names)
    # print(rows)
    for row in rows:
      
      origin_date = datetime.strptime(row[2],"%Y-%m-%d %H:%M:%S")
      # print(datetime.now() - timedelta(days=3))
      # origin date should be atleast than 3 days to the requested date
      if(datetime.now() - timedelta(days=2) <= origin_date):
        trains[names[0]] = row[0]
        trains[names[1]] = row[1]
        trains[names[2]] = origin_date
        trains[names[3]] = datetime.strptime(row[3],"%Y-%m-%d %H:%M:%S")
        trains[names[4]] = row[4]
        trains[names[5]] = row[5]
        trains[names[6]] = row[6]
        trains[names[7]] = row[7]
        trains[names[8]] = row[8]
        trains[names[9]] = row[9]

        trains[names[10]] = row[10]
        trains[names[11]] = row[11]
        trains[names[12]] = row[12]
        trains[names[13]] = row[13]
        trainObj.append(trains)
    print(trainObj)
    # rows =  [list(i) for i in rows]
    
    # names.remove("table_constraints")

    # a=pd.DataFrame( rows, columns=names)
    # print(rows)
    return trainObj
  except Exception as e:
    print(e)
    return None

def getFreightCarId():
  """
  Returns a freight car type Id 
  @return: car_type_id
  """
  sql = "select car_type_id from car_types where car_type like '%flatcar%' order by car_type_id DESC limit 1"
  cursor.execute(sql)
  conn.commit()
  car = cursor.fetchone()
  if car and len(car) > 0:
    return car[0]
  return None


def scheduleTrain():
  """
    To Schedule a train
    @params: source: String, destination: String,  
    @return: List<Train Journey> 
  """
  # User's menu and options would be displayed here
  trains = find_train("Stavanger", "Kristiansand", "2020-08-12 12:16:55")

  # print(cust_id)
  # print(trains[0].get("journey_id"))

  if (not trains) or len(trains) == 0:
    return None

  cust_id = add_customer(("Thomas Thaulow","Tjernsrudveien 43a","Stabekk","Viken","1368","40049254"))
  # print(trains)
  if create_journey_customer((cust_id,trains[0].get("journey_id"))):
    return "Scheduled Successfully!"

  return None
  

In [6]:
scheduleTrain()
# trains = find_train("Stavanger", "Kristiansand", "2020-08-12 12:16:55")
# trains

[{'origin_tName': 'Stavanger', 'dest_tName': 'Kristiansand', 'departure_datetime': datetime.datetime(2020, 8, 12, 12, 16, 55), 'arrival_datetime': datetime.datetime(2020, 8, 12, 21, 4, 20), 'car_id': 1, 'car_type_id': 1, 'car_weight': 50, 'car_capacity': 50000.0, 'car_current_capacity': 0.0, 'car_length': '49', 'car_builddate': '2020-08-12', 'table_constraints': None, 'car_type': 'flatcar', 'journey_id': 1}]


'Scheduled Successfully!'

#### STORED PROCEDURE 2 ####

In [7]:
def schedule_shipment(shipment,cust_id):
  """
    To Schedule a shipment
    @params: cust_id: integer, shipment: Tuple(cust_id, shipment_item, shipment_description, shipment_weight, shipment_volume, shipment_price),  
    @return: List<Shipment Journey>
  """
  try:
    freightCarSql = """ 
      Select trains.loc_id,trains.train_id,journey_cust_id,cars.* from journey_customer 
      left join train_journey on train_journey.journey_id = journey_customer.journey_id
      left join trains on trains.train_id = train_journey.train_id
      left join cars on cars.car_id = trains.car_id
      left join car_types on car_types.car_type_id = cars.car_type_id
      where car_type like '%flatcar%' and cust_id = ? and cars.car_current_capacity != cars.car_capacity   
      order by cars.car_current_capacity ASC
    """
    cursor.execute(freightCarSql,(cust_id,))
    conn.commit()
    car = cursor.fetchone()
    carObj = {}
    names = [ x[0] for x in cursor.description]
    for i in range(0,len(names)):
      carObj[names[i]] = car[i]
    print(carObj)

    ship_id = create_shipments(shipment)
    car_id = carObj.get("car_id")

    # Checking out the current capacity of the train
    if (carObj.get('car_capacity') >= carObj.get('car_current_capacity')+shipment[3]):
      ship_journey = create_shipment_journey((cust_id,carObj.get('train_id'),ship_id))
      return ship_journey
    else:
      car_type_id =  getFreightCarId()
      if car_type_id:
        car_id = create_cars((car_type_id,50,50000,49,"2018-02-15"))
        train_id = create_train(train(carObj.get('loc_id',car_id))) 
        ship_journey = create_shipment_journey((cust_id,train_id,ship_id))
        return ship_journey
      else:
        return None
  except Exception as e:
    print(e)
    return None

In [8]:
schedule_shipment(("Gold", "Lots of gold", 200, 40, 2500),1)

{'loc_Id': 1, 'train_id': 1, 'journey_cust_id': 1, 'car_id': 1, 'car_type_id': 1, 'car_weight': 50, 'car_capacity': 50000.0, 'car_current_capacity': 0.0, 'car_length': '49', 'car_builddate': '2020-08-12', 'table_constraints': None}


1

# 4- Queries & Reporting #

##### REPORT 1 #####

In [9]:
# EXAMPLE TERMINAL NAME: Kristiansand 
# EXAMPLE REPORT_DATE 2020-08-12 21:04:20

terminal_selection = ("Kristiansand")
#terminal_selection = input("Enter terminal name: ")


## Shows report for any chosen day. Can also show daily arrivals by uncommenting "report_date"
date_today = datetime.now().strftime('%d-%m-%y%y')
#report_date = input("add your input, DD-MM-YYYY: ")
report_date = ("2020-08-12 21:04:20")


schema = """
    SELECT TR.train_id as TrainID, t2.terminal_name as ArrivalName, t1.terminal_name AS Origin, D.arrival_datetime AS ArrivalTime, COUNT(TR.car_id) AS Cars
    FROM trains TR,origin O, destination D, train_journey TJ, terminals T
    LEFT JOIN terminals as t2 on D.terminal_id = t2.terminal_id 
    LEFT JOIN terminals as t1 on O.terminal_id = t1.terminal_id 
    WHERE TR.train_id = TJ.train_id
    AND t2.terminal_name = ?
    AND D.arrival_datetime = ?
    GROUP BY TR.train_id
    """

cursor = conn.cursor()
cursor.execute(schema,(terminal_selection,report_date))
names = [ x[0] for x in cursor.description]
rows = cursor.fetchall()
main_table = pd.DataFrame( rows, columns=names)
main_table

,TrainID,ArrivalName,Origin,ArrivalTime,Cars
0,1,Kristiansand,Stavanger,2020-08-12 21:04:20,3


#### REPORT 2 ####

In [10]:
# EXAMPLE DEPARTURE TIME: 2020-08-12 12:16:55
# EXAMPLE ARRIVAL TIME:  2020-08-12 21:04:20

start_date = input("Enter start date: ")
end_date = input("Enter end date: ")

schema = """
    SELECT customers.cust_name as Name, customers.cust_phone as Phone, sum(shipments.shipment_weight) AS Weight
    FROM customers
    LEFT JOIN journey_customer on  customers.cust_id = journey_customer.journey_cust_id 
    LEFT JOIN train_journey on train_journey.journey_id = journey_customer.journey_id
    LEFT JOIN origin on train_journey.origin_id = origin.origin_id
    LEFT JOIN destination on train_journey.destination_id = destination.destination_id
    Left JOIN shipment_journey on  journey_customer.journey_cust_id = shipment_journey.journey_cust_id
    LEFT JOIN shipments on  shipment_journey.shipment_id = shipments.shipment_id 
    WHERE origin.departure_datetime
    BETWEEN ? AND ?
    OR destination.arrival_datetime
    BETWEEN ? AND ?
    GROUP BY customers.cust_id
    ORDER BY shipments.shipment_weight DESC
    """

cursor = conn.cursor()
cursor.execute(schema,(start_date, end_date, start_date, end_date))
names = [ x[0] for x in cursor.description]
rows = cursor.fetchall()
main_table = pd.DataFrame( rows, columns=names)
main_table

Enter start date: 2020-08-12 12:16:55
Enter end date: 2020-08-12 12:16:55


,Name,Phone,Weight
0,Thomas Thaulow,40049254,200
